![Logo BV IBMEC](https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/logo-bv-ibmec-notebooks.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ian-iania/IBMEC-BV-Modelos-Preditivos/blob/main/notebooks/NB03_Survival.ipynb)

# NB03 — Survival Analysis: Prepay e Duração da Carteira (Auto com LTV)

- entender evento “quando?” com censura
- ler curva Kaplan–Meier (S(t)) de forma executiva
- relacionar Selic + LTV com duração e impacto no planejamento/NII proxy

### Padrão visual deste NB03

- Neste NB03 usamos **Plotly** para curvas e comparações (KM global, grupos e validação temporal).
- Usamos **Matplotlib** apenas para histogramas simples.
- Se algum gráfico não aparecer no Colab, reinicie o runtime e rode o notebook novamente.

## 1) Contexto FP&A

### O que vamos fazer
Conectar a lógica de Survival com decisões de planejamento em carteira Auto.

### Por que isso importa para FP&A
- Regressão responde **quanto**.
- Classificação responde **se**.
- Survival responde **quando** o evento acontece.

### O que observar
Prepay mais rápido encurta duração da carteira, reduz juros futuros e pode afetar NII.

## 1.1) Instalar biblioteca de survival

### O que vamos fazer
Instalar `lifelines` para ajustar Kaplan-Meier.

### Por que isso importa para FP&A
Sem essa biblioteca não conseguimos gerar a curva de sobrevivência de forma padronizada.

### O que observar
A célula pode demorar um pouco no Colab, mas deve concluir sem erro.

In [ ]:
!pip -q install lifelines

## 1.2) Imports da aula

### O que vamos fazer
Carregar bibliotecas para dados, gráficos e Survival.

### Por que isso importa para FP&A
Essas bibliotecas sustentam toda a análise: limpeza, métricas e visual executivo.

### O que observar
Sem output complexo; apenas preparação do ambiente.

In [ ]:
import pandas as pd  # manipulação de tabelas e datas
import numpy as np  # operações numéricas
import matplotlib.pyplot as plt  # histogramas simples
import plotly.express as px  # gráficos interativos prontos
import plotly.graph_objects as go  # curvas customizadas no padrão executivo
from lifelines import KaplanMeierFitter  # estimador Kaplan-Meier

## 2) Carregar dataset e fazer sanidade inicial

### O que vamos fazer
Ler a base sintética de prepay pelo GitHub raw, converter data e ordenar no tempo.

### Por que isso importa para FP&A
Sem ordem temporal correta, a leitura de duration e validação pode ficar distorcida.

### O que observar
Confirme o tamanho da base e as primeiras linhas já ordenadas por `orig_dt`.

In [ ]:
url = "https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_auto_prepay_survival_sintetico.csv"  # fonte oficial da base

df = pd.read_csv(url, parse_dates=["orig_dt"])  # lê a base e já converte data de originação
df = df.sort_values("orig_dt").reset_index(drop=True)  # garante ordem temporal crescente

### 2.1) Primeira leitura da base

### O que vamos fazer
Ver tamanho, início da base e taxa de evento/censura.

### Por que isso importa para FP&A
Essa checagem rápida evita seguir com suposições erradas sobre evento observado.

### O que observar
- taxa de `E_prepay=1` (evento)
- taxa de `E_prepay=0` (censura)

In [ ]:
print(f"Shape da base: {df.shape[0]} linhas x {df.shape[1]} colunas")  # mostra tamanho da base

event_rate = df["E_prepay"].mean()  # calcula taxa de evento observado
censor_rate = 1 - event_rate  # calcula taxa de censura

print(f"Taxa de evento (E_prepay=1): {event_rate:.2%}")  # imprime taxa de evento
print(f"Taxa de censura (E_prepay=0): {censor_rate:.2%}")  # imprime taxa de censura

df.head(3)  # exibe primeiras linhas para checagem visual

### 2.2) Histograma de duração observada (`T_meses`)

### O que vamos fazer
Plotar a distribuição de tempo observado até evento/censura.

### Por que isso importa para FP&A
Ajuda a entender se a carteira concentra saídas cedo ou mantém duração mais longa.

### O que observar
A concentração do histograma indica onde está a maior massa de contratos.

In [ ]:
plt.figure(figsize=(8, 4))  # abre figura do histograma
plt.hist(df["T_meses"], bins=20, alpha=0.85)  # distribui contratos por meses observados
plt.title("Distribuição de T_meses")  # define título executivo
plt.xlabel("T_meses")  # nome do eixo x
plt.ylabel("Quantidade de contratos")  # nome do eixo y
plt.tight_layout()  # ajusta layout
plt.show()  # renderiza gráfico

**Leitura FP&A:** se a massa estiver concentrada em meses baixos, a carteira gira mais rápido e a duration tende a cair.

### 2.3) Histograma de LTV

### O que vamos fazer
Ver como o LTV está distribuído na base.

### Por que isso importa para FP&A
LTV é um driver clássico de risco e pode alterar velocidade de prepay em alguns contextos.

### O que observar
Compare visualmente faixas de LTV baixo vs alto para usar nas comparações de KM.

In [ ]:
plt.figure(figsize=(8, 4))  # abre figura para distribuição de LTV
plt.hist(df["ltv"], bins=20, alpha=0.85)  # plota histograma do LTV
plt.title("Distribuição de LTV")  # define título do gráfico
plt.xlabel("LTV")  # eixo x
plt.ylabel("Quantidade de contratos")  # eixo y
plt.tight_layout()  # ajusta margens
plt.show()  # renderiza gráfico

**Leitura FP&A:** essa distribuição ajuda a definir cortes didáticos para “LTV baixo” e “LTV alto” no restante da aula.

### 2.4) Selic ao longo do tempo de originação

### O que vamos fazer
Agregaremos Selic média por mês de originação para enxergar contexto macro.

### Por que isso importa para FP&A
Mudança de regime macro costuma alterar comportamento de prepay e duração.

### O que observar
Tendência e mudanças de nível da Selic ao longo dos meses.

In [ ]:
selic_mensal = (  # agrega Selic média por mês
    df.set_index("orig_dt")["selic_at_orig"]
      .resample("MS")
      .mean()
      .reset_index()
)

fig_selic = px.line(  # cria linha de Selic no tempo
    selic_mensal,
    x="orig_dt",
    y="selic_at_orig",
    title="PARTE A — Selic média por mês de originação"
)
fig_selic.update_layout(xaxis_title="Originação", yaxis_title="Selic na originação")  # nomeia eixos
fig_selic.show()  # exibe gráfico interativo

**Leitura FP&A:** quando o macro muda, o padrão de comportamento da carteira também pode mudar. Guardaremos isso para a validação temporal.

## 3) Conceitos mínimos: T, E e censura

- `T_meses`: tempo observado até evento ou fim da observação.
- `E_prepay`: indicador do evento (`1` ocorreu prepay, `0` censurado).
- **Censura não é zero risco**; significa apenas “evento ainda não observado até o corte”.

## 4) Kaplan-Meier global (curva executiva)

### O que vamos fazer
Ajustar o KM da carteira inteira e preparar métricas executivas.

### Por que isso importa para FP&A
KM entrega uma leitura clara da duração esperada sem exigir modelagem paramétrica complexa.

### O que observar
Depois veremos `S(12)`, `S(24)`, mediana e RMST36.

In [ ]:
km_global = KaplanMeierFitter()  # instancia o estimador Kaplan-Meier
km_global.fit(df["T_meses"], event_observed=df["E_prepay"], label="Carteira total")  # ajusta curva global

### 4.1) Criar utilitários de métricas de survival

### O que vamos fazer
Criar funções curtas para padronizar `S(12)`, `S(24)`, mediana e RMST36.

### Por que isso importa para FP&A
Padronizar cálculo evita inconsistência na comparação entre grupos.

### O que observar
As mesmas métricas serão usadas em todas as tabelas executivas.

In [ ]:
def km_curve_df(kmf, group_name):  # transforma curva KM em DataFrame para plot
    out = kmf.survival_function_.reset_index()  # converte série de sobrevivência em tabela
    out.columns = ["t", "S(t)"]  # renomeia colunas para leitura didática
    out["grupo"] = group_name  # adiciona nome do grupo na tabela
    return out  # devolve tabela da curva


def rmst36(kmf):  # calcula RMST até 36 meses
    timeline = np.arange(0, 37)  # define grade de tempo de 0 a 36 meses
    surv = kmf.predict(timeline).values  # obtém S(t) em cada mês da grade
    return float(np.trapezoid(surv, timeline))  # integra área sob a curva


def km_summary_row(kmf, group_name):  # gera linha executiva de métricas
    median_val = kmf.median_survival_time_  # lê mediana da sobrevivência
    median_val = np.nan if np.isinf(median_val) else float(median_val)  # trata caso sem mediana finita
    return {
        "grupo": group_name,
        "S(12)": float(kmf.predict(12)),
        "S(24)": float(kmf.predict(24)),
        "mediana_meses": median_val,
        "RMST36": rmst36(kmf)
    }

### 4.2) Plotar curva KM global (Plotly)

### O que vamos fazer
Transformar a curva em gráfico de degraus para leitura executiva.

### Por que isso importa para FP&A
Em survival, o degrau mostra claramente a queda da sobrevivência ao longo do tempo.

### O que observar
Vamos destacar `S(12)` e `S(24)` no gráfico.

In [ ]:
km_global_df = km_curve_df(km_global, "Carteira total")  # monta DataFrame da curva global

s12_global = float(km_global.predict(12))  # captura sobrevivência no mês 12
s24_global = float(km_global.predict(24))  # captura sobrevivência no mês 24

fig_km_global = go.Figure()  # inicia figura Plotly
fig_km_global.add_trace(  # adiciona curva em degraus da carteira
    go.Scatter(
        x=km_global_df["t"],
        y=km_global_df["S(t)"],
        mode="lines",
        line_shape="hv",
        name="KM global"
    )
)

### 4.2.1) Destacar pontos de referência (12 e 24 meses)

### O que vamos fazer
Adicionar marcações e título executivo ao gráfico global.

### Por que isso importa para FP&A
Pontos fixos facilitam comunicação em fóruns executivos.

### O que observar
Os valores de `S(12)` e `S(24)` aparecem direto na figura.

In [ ]:
fig_km_global.add_trace(  # marca ponto S(12)
    go.Scatter(x=[12], y=[s12_global], mode="markers+text", text=[f"S(12)={s12_global:.2%}"], textposition="top right", name="S(12)")
)
fig_km_global.add_trace(  # marca ponto S(24)
    go.Scatter(x=[24], y=[s24_global], mode="markers+text", text=[f"S(24)={s24_global:.2%}"], textposition="top right", name="S(24)")
)
fig_km_global.update_layout(  # configura layout executivo
    title="PARTE B — Kaplan-Meier global da carteira",
    xaxis_title="Tempo (meses)",
    yaxis_title="S(t) - probabilidade de permanecer sem prepay"
)
fig_km_global.show()  # exibe figura

**Leitura FP&A:** quanto mais rápido a curva cai, menor a duração da carteira. Pontos S(12) e S(24) ajudam na comunicação executiva.

### 4.3) Tabela executiva da curva global

### O que vamos fazer
Consolidar `S(12)`, `S(24)`, mediana e RMST36 em uma mini-tabela.

### Por que isso importa para FP&A
Esse quadro é ideal para resumo de comitê e planejamento.

### O que observar
A RMST36 representa duração média esperada até 36 meses.

In [ ]:
summary_global = pd.DataFrame([km_summary_row(km_global, "Carteira total")])  # cria tabela com métricas globais
summary_global  # exibe resumo executivo

## 5) KM por Selic: sensibilidade macro (Q1 vs Q4)

### O que vamos fazer
Comparar extremos de Selic na originação (quartil baixo vs quartil alto).

### Por que isso importa para FP&A
Mostra como regime macro pode acelerar ou desacelerar a duration.

### O que observar
Curva que cai mais rápido indica maior ritmo de prepay.

In [ ]:
q1_selic = df["selic_at_orig"].quantile(0.25)  # calcula limite do quartil inferior
q4_selic = df["selic_at_orig"].quantile(0.75)  # calcula limite do quartil superior

df_selic = df[(df["selic_at_orig"] <= q1_selic) | (df["selic_at_orig"] >= q4_selic)].copy()  # mantém apenas extremos

df_selic["grupo_selic"] = np.where(df_selic["selic_at_orig"] <= q1_selic, "Selic baixa (Q1)", "Selic alta (Q4)")  # rotula grupos

### 5.1) Ajustar KM por grupo de Selic

### O que vamos fazer
Ajustar uma curva para cada grupo e preparar tabela comparativa.

### Por que isso importa para FP&A
Comparação lado a lado facilita discutir mudança de comportamento por cenário macro.

### O que observar
Depois veremos curvas e métricas no mesmo padrão.

In [ ]:
km_selic_low = KaplanMeierFitter()  # instancia KM para grupo de Selic baixa
km_selic_high = KaplanMeierFitter()  # instancia KM para grupo de Selic alta

mask_low_selic = df_selic["grupo_selic"] == "Selic baixa (Q1)"  # filtro do grupo baixo
mask_high_selic = df_selic["grupo_selic"] == "Selic alta (Q4)"  # filtro do grupo alto

km_selic_low.fit(df_selic.loc[mask_low_selic, "T_meses"], event_observed=df_selic.loc[mask_low_selic, "E_prepay"], label="Selic baixa (Q1)")  # ajusta KM do grupo baixo
km_selic_high.fit(df_selic.loc[mask_high_selic, "T_meses"], event_observed=df_selic.loc[mask_high_selic, "E_prepay"], label="Selic alta (Q4)")  # ajusta KM do grupo alto

### 5.2) Plotar curvas KM de Selic (Plotly)

### O que vamos fazer
Exibir as duas curvas no mesmo gráfico para comparação direta.

### Por que isso importa para FP&A
É a forma mais rápida de comunicar sensibilidade de duração ao macro.

### O que observar
Distância vertical entre curvas e velocidade de queda.

In [ ]:
curve_selic_low = km_curve_df(km_selic_low, "Selic baixa (Q1)")  # tabela da curva de Selic baixa
curve_selic_high = km_curve_df(km_selic_high, "Selic alta (Q4)")  # tabela da curva de Selic alta

fig_selic_km = go.Figure()  # inicia figura comparativa
fig_selic_km.add_trace(go.Scatter(x=curve_selic_low["t"], y=curve_selic_low["S(t)"], mode="lines", line_shape="hv", name="Selic baixa (Q1)"))  # adiciona curva baixa
fig_selic_km.add_trace(go.Scatter(x=curve_selic_high["t"], y=curve_selic_high["S(t)"], mode="lines", line_shape="hv", name="Selic alta (Q4)"))  # adiciona curva alta
fig_selic_km.update_layout(title="PARTE C — KM por grupo de Selic", xaxis_title="Tempo (meses)", yaxis_title="S(t)")  # define títulos
fig_selic_km.show()  # exibe gráfico

### 5.3) Tabela executiva por Selic

### O que vamos fazer
Consolidar métricas de survival para os dois grupos.

### Por que isso importa para FP&A
Permite transformar gráfico em números para decisão.

### O que observar
Compare S(12), S(24), mediana e RMST36 entre os grupos.

In [ ]:
summary_selic = pd.DataFrame([
    km_summary_row(km_selic_low, "Selic baixa (Q1)"),
    km_summary_row(km_selic_high, "Selic alta (Q4)")
])  # gera tabela de métricas por Selic

summary_selic  # exibe tabela executiva

**Leitura FP&A:** grupo com menor RMST36 tende a ter duration mais curta, exigindo ajuste de planejamento de margem no horizonte.

## 6) KM por LTV: sensibilidade de estrutura da operação

### O que vamos fazer
Comparar dois grupos didáticos: LTV baixo (`<= 0.75`) e LTV alto (`>= 0.95`).

### Por que isso importa para FP&A
LTV é driver central em Auto e pode alterar comportamento de permanência da carteira.

### O que observar
Curva mais baixa ao longo do tempo indica menor sobrevivência sem prepay.

In [ ]:
df_ltv = df[(df["ltv"] <= 0.75) | (df["ltv"] >= 0.95)].copy()  # filtra extremos didáticos de LTV

df_ltv["grupo_ltv"] = np.where(df_ltv["ltv"] <= 0.75, "LTV baixo (<=0.75)", "LTV alto (>=0.95)")  # cria rótulos dos grupos

### 6.1) Ajustar KM para os grupos de LTV

### O que vamos fazer
Ajustar Kaplan-Meier separado para LTV baixo e alto.

### Por que isso importa para FP&A
Compara comportamento de duração por perfil da operação.

### O que observar
Esses resultados serão levados para a tabela executiva final.

In [ ]:
km_ltv_low = KaplanMeierFitter()  # instancia KM para LTV baixo
km_ltv_high = KaplanMeierFitter()  # instancia KM para LTV alto

mask_ltv_low = df_ltv["grupo_ltv"] == "LTV baixo (<=0.75)"  # cria máscara do grupo baixo
mask_ltv_high = df_ltv["grupo_ltv"] == "LTV alto (>=0.95)"  # cria máscara do grupo alto

km_ltv_low.fit(df_ltv.loc[mask_ltv_low, "T_meses"], event_observed=df_ltv.loc[mask_ltv_low, "E_prepay"], label="LTV baixo (<=0.75)")  # ajusta KM do grupo baixo
km_ltv_high.fit(df_ltv.loc[mask_ltv_high, "T_meses"], event_observed=df_ltv.loc[mask_ltv_high, "E_prepay"], label="LTV alto (>=0.95)")  # ajusta KM do grupo alto

### 6.2) Plotar KM por LTV (Plotly)

### O que vamos fazer
Exibir as duas curvas de LTV no mesmo gráfico interativo.

### Por que isso importa para FP&A
Facilita explicar para negócio como estrutura de carteira afeta duração.

### O que observar
A diferença de inclinação mostra quem desacumula mais rápido.

In [ ]:
curve_ltv_low = km_curve_df(km_ltv_low, "LTV baixo (<=0.75)")  # curva do grupo LTV baixo
curve_ltv_high = km_curve_df(km_ltv_high, "LTV alto (>=0.95)")  # curva do grupo LTV alto

fig_ltv_km = go.Figure()  # inicia figura de comparação
fig_ltv_km.add_trace(go.Scatter(x=curve_ltv_low["t"], y=curve_ltv_low["S(t)"], mode="lines", line_shape="hv", name="LTV baixo (<=0.75)"))  # adiciona curva baixa
fig_ltv_km.add_trace(go.Scatter(x=curve_ltv_high["t"], y=curve_ltv_high["S(t)"], mode="lines", line_shape="hv", name="LTV alto (>=0.95)"))  # adiciona curva alta
fig_ltv_km.update_layout(title="PARTE D — KM por grupo de LTV", xaxis_title="Tempo (meses)", yaxis_title="S(t)")  # define layout
fig_ltv_km.show()  # mostra gráfico

### 6.3) Tabela executiva por LTV

### O que vamos fazer
Consolidar as métricas no mesmo formato de Selic.

### Por que isso importa para FP&A
Padronização facilita comparação entre hipóteses macro e carteira.

### O que observar
Diferenças de mediana e RMST36 entre LTV baixo e alto.

In [ ]:
summary_ltv = pd.DataFrame([
    km_summary_row(km_ltv_low, "LTV baixo (<=0.75)"),
    km_summary_row(km_ltv_high, "LTV alto (>=0.95)")
])  # monta quadro de métricas por LTV

summary_ltv  # exibe tabela comparativa

**Leitura FP&A:** diferenças estruturais de LTV podem alterar o tempo médio de permanência e, portanto, projeções de receita no tempo.

## 7) KM por spread (alto vs baixo)

### O que vamos fazer
Usar `rate_spread_m` para comparar grupos de spread baixo e spread alto.

### Por que isso importa para FP&A
Spread conversa diretamente com margem e pode estar associado a comportamento distinto de prepay.

### O que observar
Curvas e métricas mostram se há diferença relevante de duration por spread.

In [ ]:
q1_spread = df["rate_spread_m"].quantile(0.25)  # define limite do spread baixo
q4_spread = df["rate_spread_m"].quantile(0.75)  # define limite do spread alto

df_spread = df[(df["rate_spread_m"] <= q1_spread) | (df["rate_spread_m"] >= q4_spread)].copy()  # seleciona extremos

df_spread["grupo_spread"] = np.where(df_spread["rate_spread_m"] <= q1_spread, "Spread baixo (Q1)", "Spread alto (Q4)")  # cria rótulos dos grupos

km_spread_low = KaplanMeierFitter()  # instancia KM para spread baixo
km_spread_high = KaplanMeierFitter()  # instancia KM para spread alto

km_spread_low.fit(df_spread.loc[df_spread["grupo_spread"] == "Spread baixo (Q1)", "T_meses"], event_observed=df_spread.loc[df_spread["grupo_spread"] == "Spread baixo (Q1)", "E_prepay"], label="Spread baixo (Q1)")  # ajusta grupo baixo
km_spread_high.fit(df_spread.loc[df_spread["grupo_spread"] == "Spread alto (Q4)", "T_meses"], event_observed=df_spread.loc[df_spread["grupo_spread"] == "Spread alto (Q4)", "E_prepay"], label="Spread alto (Q4)")  # ajusta grupo alto

### 7.1) Plotar KM por spread (Plotly)

### O que vamos fazer
Comparar visualmente as curvas de spread no mesmo eixo.

### Por que isso importa para FP&A
Ajuda a construir narrativa sobre margem e comportamento de carteira.

### O que observar
Nível e inclinação de cada curva ao longo do horizonte.

In [ ]:
curve_spread_low = km_curve_df(km_spread_low, "Spread baixo (Q1)")  # prepara curva de spread baixo
curve_spread_high = km_curve_df(km_spread_high, "Spread alto (Q4)")  # prepara curva de spread alto

fig_spread_km = go.Figure()  # inicia figura de spread
fig_spread_km.add_trace(go.Scatter(x=curve_spread_low["t"], y=curve_spread_low["S(t)"], mode="lines", line_shape="hv", name="Spread baixo (Q1)"))  # adiciona curva baixa
fig_spread_km.add_trace(go.Scatter(x=curve_spread_high["t"], y=curve_spread_high["S(t)"], mode="lines", line_shape="hv", name="Spread alto (Q4)"))  # adiciona curva alta
fig_spread_km.update_layout(title="PARTE E — KM por grupo de spread", xaxis_title="Tempo (meses)", yaxis_title="S(t)")  # configura layout
fig_spread_km.show()  # renderiza gráfico

### 7.2) Tabela executiva por spread

### O que vamos fazer
Apresentar métricas para os grupos de spread no mesmo padrão.

### Por que isso importa para FP&A
Mantém consistência para discussão de impacto de pricing.

### O que observar
Diferença em RMST36 e S(24) entre spreads baixos e altos.

In [ ]:
summary_spread = pd.DataFrame([
    km_summary_row(km_spread_low, "Spread baixo (Q1)"),
    km_summary_row(km_spread_high, "Spread alto (Q4)")
])  # gera resumo por spread

summary_spread  # exibe tabela

**Leitura FP&A:** curvas por spread ajudam a traduzir trade-offs entre margem e velocidade de saída da carteira.

## 8) Impacto em NII (proxy simples)

### O que vamos fazer
Calcular um proxy didático: `NII_proxy_36m = RMST36 x margem_mensal_média`.

### Por que isso importa para FP&A
Conecta survival com planejamento financeiro sem entrar em modelagem financeira pesada.

### O que observar
Grupos com maior RMST36 e boa margem tendem a ter NII proxy maior.

In [ ]:
def group_exec_row(sub_df, group_name):  # calcula métricas executivas de um grupo
    km_group = KaplanMeierFitter()  # instancia KM para o grupo
    km_group.fit(sub_df["T_meses"], event_observed=sub_df["E_prepay"], label=group_name)  # ajusta curva

    row = km_summary_row(km_group, group_name)  # reaproveita métricas padrão de survival
    row["n_contratos"] = int(sub_df.shape[0])  # adiciona volume do grupo
    row["margem_mensal_media"] = float(sub_df["margin_monthly_proxy"].mean())  # adiciona margem média mensal
    row["NII_proxy_36m"] = row["RMST36"] * row["margem_mensal_media"]  # calcula proxy de NII por contrato
    return row  # devolve linha pronta

### 8.1) Exibir tabela executiva de NII proxy

### O que vamos fazer
Gerar o quadro final com métricas de survival e proxy de NII por grupo.

### Por que isso importa para FP&A
É o elo entre comportamento de risco (quando) e impacto financeiro (R$).

### O que observar
Compare principalmente `RMST36` e `NII_proxy_36m`.

In [ ]:
group_frames = {  # organiza grupos que entrarão no quadro executivo
    "Global": df,
    "Selic baixa (Q1)": df_selic[df_selic["grupo_selic"] == "Selic baixa (Q1)"],
    "Selic alta (Q4)": df_selic[df_selic["grupo_selic"] == "Selic alta (Q4)"],
    "LTV baixo (<=0.75)": df_ltv[df_ltv["grupo_ltv"] == "LTV baixo (<=0.75)"],
    "LTV alto (>=0.95)": df_ltv[df_ltv["grupo_ltv"] == "LTV alto (>=0.95)"]
}

rows_exec = [group_exec_row(frame, name) for name, frame in group_frames.items()]  # calcula métricas para cada grupo
table_exec = pd.DataFrame(rows_exec)  # consolida o resumo executivo

table_exec  # exibe quadro final de grupos

### 8.2) Plotar NII proxy por grupo (Plotly)

### O que vamos fazer
Visualizar, em barras, o NII proxy por grupo comparável.

### Por que isso importa para FP&A
Barras ajudam a comunicar rápido o impacto relativo entre cenários/grupos.

### O que observar
A ordem das barras indica quais grupos têm maior potencial de permanência x margem.

In [ ]:
fig_nii = px.bar(  # cria gráfico de barras do NII proxy
    table_exec,
    x="grupo",
    y="NII_proxy_36m",
    title="PARTE F — NII proxy 36m por grupo (R$ por contrato)",
    text="NII_proxy_36m"
)
fig_nii.update_layout(xaxis_title="Grupo", yaxis_title="NII proxy 36m (R$)")  # define eixos
fig_nii.show()  # exibe gráfico

**Leitura FP&A:** este proxy não substitui projeção completa de NII, mas ajuda a priorizar discussões de duração e margem.

## 9) Validação temporal: treino vs teste

### O que vamos fazer
Separar originação em treino (até 2022-12-31) e teste (2023–2024) para comparar curvas.

### Por que isso importa para FP&A
Comportamento muda com regime macro; validação temporal evita falsa segurança.

### O que observar
Diferença entre curvas e métricas nos dois períodos.

In [ ]:
cutoff_date = pd.Timestamp("2022-12-31")  # define data de corte temporal

train_df = df[df["orig_dt"] <= cutoff_date].copy()  # separa amostra de treino no tempo
test_df = df[df["orig_dt"] > cutoff_date].copy()  # separa amostra de teste no tempo

km_train = KaplanMeierFitter().fit(train_df["T_meses"], event_observed=train_df["E_prepay"], label="Treino (<=2022-12)")  # ajusta KM de treino
km_test = KaplanMeierFitter().fit(test_df["T_meses"], event_observed=test_df["E_prepay"], label="Teste (2023-2024)")  # ajusta KM de teste

### 9.1) Curvas KM treino vs teste (Plotly)

### O que vamos fazer
Plotar as duas curvas no mesmo gráfico para avaliar mudança de regime.

### Por que isso importa para FP&A
Se teste diverge do treino, precisamos monitorar e recalibrar expectativas.

### O que observar
Distância entre as curvas e velocidade de queda no período recente.

In [ ]:
curve_train = km_curve_df(km_train, "Treino")  # cria tabela de curva do treino
curve_test = km_curve_df(km_test, "Teste")  # cria tabela de curva do teste

fig_val = go.Figure()  # inicia figura de validação temporal
fig_val.add_trace(go.Scatter(x=curve_train["t"], y=curve_train["S(t)"], mode="lines", line_shape="hv", name="Treino (<=2022-12)"))  # adiciona curva treino
fig_val.add_trace(go.Scatter(x=curve_test["t"], y=curve_test["S(t)"], mode="lines", line_shape="hv", name="Teste (2023-2024)"))  # adiciona curva teste
fig_val.update_layout(title="PARTE G — Validação temporal (KM treino vs teste)", xaxis_title="Tempo (meses)", yaxis_title="S(t)")  # configura layout
fig_val.show()  # exibe figura

### 9.2) Tabela comparativa treino vs teste

### O que vamos fazer
Consolidar métricas executivas para os dois períodos.

### Por que isso importa para FP&A
A tabela ajuda a monitorar estabilidade do comportamento da carteira.

### O que observar
Diferença em `S(12)`, `S(24)`, mediana e RMST36.

In [ ]:
summary_time = pd.DataFrame([
    km_summary_row(km_train, "Treino (<=2022-12)"),
    km_summary_row(km_test, "Teste (2023-2024)")
])  # monta quadro comparativo temporal

summary_time  # exibe tabela de validação temporal

**Leitura FP&A:** diferença relevante entre treino e teste sugere mudança de regime e necessidade de monitoramento recorrente.

## 10) Fechamento executivo + checklist FP&A

### Conclusões estilo comitê
- Survival responde **quando** o evento ocorre, com linguagem clara para negócio.
- Curva KM global oferece leitura direta de duração via `S(12)`, `S(24)` e mediana.
- Sensibilidade por Selic e LTV mostra como macro e estrutura alteram tempo de permanência.
- RMST36 e NII proxy ajudam a traduzir duration em impacto financeiro para planejamento.
- Validação temporal é obrigatória para evitar decisões baseadas em regime antigo.
- Governance: documentar premissas, acompanhar drift e revisar periodicamente.

### Checklist FP&A (1 minuto)
- Baseline de survival: curva KM global calculada.
- Split temporal aplicado (treino vs teste).
- Censura interpretada corretamente (`E=0` não é “zero risco”).
- Métricas executivas reportadas (`S(12)`, `S(24)`, mediana, RMST36).
- Sensibilidade macro/portfolio avaliada (Selic, LTV, spread).
- Monitoramento contínuo e revisão de premissas em mudança de regime.